# DL vs logistic regression for a standard classification problem

Bla bla bla...

## Set up

In [ ]:
using Pkg
Pkg.status()
Pkg.build("Knet")
using Knet
using PyPlot
using LinearAlgebra
using Random
using Printf
import StatsBase.predict
using GLM
using DataFrames

include(pwd()*"/utilities.jl")

## DL model

Load data.

In [ ]:
Random.seed!(1234) # set random numbers
x_train,y_train,x_val,y_val,x_test,y_test = load_data([1,2])

Set up network

In [ ]:
n_input = 8
n_hidden_1 = 100
n_hidden_2 = 50
n_hidden_3 = 25
n_out = 2

w = Any[ xavier(n_hidden_1,n_input), zeros(n_hidden_1,1),
         xavier(n_hidden_2,n_hidden_1), zeros(n_hidden_2,1),
         xavier(n_hidden_3,n_hidden_2), zeros(n_hidden_3,1),
         xavier(n_out,n_hidden_3), zeros(n_out,1)]

Calc number of weights.

In [ ]:
nbr_training_obs = length(y_train)
nbr_parameters = 0

for i in w
    global nbr_parameters = nbr_parameters + size(i,1)*size(i,2)
end

@printf "Nbr training obs %d, nbr parameters %d, obs/parameters %.2f\n" nbr_training_obs nbr_parameters nbr_training_obs/nbr_parameters


Set up networks functions.

In [ ]:
function predict(w,x)
    output = zeros(2,size(x,2))
    for i=1:2:length(w)-2
        x = relu.(w[i]*x .+ w[i+1])
    end
    return w[end-1]*x .+ w[end]
end

loss(w,x,ygold) = Knet.nll(predict(w,x), ygold; average=false)

lossgradient = grad(loss)

optim = optimizers(w, Adam)

Set training function.

In [ ]:
function train(epoch::Int,dropout_percentage::Real,lambda::Vector)

    loss_train = zeros(epoch)
    loss_val = zeros(epoch)

    for i in 1:epoch

        idx_train = rand(1:size(x_train,2),size(x_train,2))
        idx_val = rand(1:size(x_val,2),size(x_val,2))

        grads = lossgradient(w,x_train[:,idx_train[:]],y_train[idx_train])
        update!(w, grads, optim)

        loss_train[i] = loss(w,x_train[:,idx_train[:]], y_train[idx_train])
        loss_val[i] = loss(w,x_val[:,idx_val], y_val[idx_val])

    end

    return loss_train, loss_val

end

Train network.

In [ ]:
@time loss_train, loss_val = train(5000,0,zeros(3))

Compute predictions.

In [ ]:
y_pred_train_proc = predict(w, x_train)
y_pred_proc = predict(w, x_test)

y_pred_train = class_nll(y_pred_train_proc,labels)
y_pred_test = class_nll(y_pred_proc,labels)

Check classfication results.

In [ ]:
# loss
loss_training = loss(w,x_train[:,end:-1:1], y_train[end:-1:1])
loss_test = loss(w,x_test[:,end:-1:1], y_test[end:-1:1])

# classification results
class_res_training = classification_results(y_pred_train, y_train, [1,2])
class_res_test = classification_results(y_pred_test, y_test, [1,2])

Print training and test loss.

In [ ]:
PyPlot.figure()
PyPlot.plot(loss_train, "b")
PyPlot.plot(loss_val, "r")

## Logistic regression model

Load data.

In [ ]:
Random.seed!(1234) # set random numbers
x_train,y_train,x_val,y_val,x_test,y_test = load_data()

Create data frames for training and test data.

In [ ]:
training_data = zeros(size(x_train,2), size(x_train,1)+1)
training_data[:,1:end-1] = x_train'
training_data[:,end] = y_train
train_df = DataFrame(training_data)
test_df = DataFrame(x_test')

Setup and train model.

In [ ]:
model = @formula(x9 ~ x1+x2+x3+x4+x5+x6+x7+x8)
@time logisticregresson = glm(model, train_df, Binomial(), LogitLink())

print(logisticregresson)

Print fitted model.

In [ ]:
print(logisticregresson)

The model fit seems to be okey since most of the parameters are significant.

Next, compute predictions for training and test data.

In [ ]:
y_pred_train = zeros(2,length(y_train))
y_pred_train[1,:] = predict(logisticregresson)
y_pred_train[2,:] = 1 .- y_pred_train[1,:]
y_pred_train = class(y_pred_train)

y_pred = zeros(2,length(y_test))
y_pred[1,:] = predict(logisticregresson,test_df)
y_pred[2,:] = 1 .- y_pred[1,:]
y_pred = class(y_pred)

Check classfication results.

In [ ]:
class_res_training = classification_results(y_pred_train, y_train)
class_res_test = classification_results(y_pred, y_test)

# print resutls nicly here!

## Comparison